This notebook is used to join some data about federal courts from Wikipedia tables to some other data from [Courtlistener](https://www.courtlistener.com/).

To use this notebook, I downloaded "All court data" from https://www.courtlistener.com/api/bulk-data/courts/all.tar.gz and put it in the a subdirectory called courts-all.

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# This scraping function could easily break if the format of the Wikipedia page changes.
# The versions of these pages as of 2018-01-06 should work.

def loadWikiPage(pagename):
    urls = {"USFedCircuitComposition" : "https://en.m.wikipedia.org/w/index.php?title=Template:USFedCircuitComposition&mobileaction=toggle_view_mobile",
            "List_of_former_United_States_district_courts": "https://en.m.wikipedia.org/wiki/List_of_former_United_States_district_courts"}
    
    pagepath = './html/' + pagename + '.html'
    
    try:
        # getting list of federal court circuits and districts from file
        f = open(pagepath)
        page = f.read()
        assert len(page) > 100
        soup = BeautifulSoup(page, 'html.parser')

    except:
        print("List of federal court circuits and districts not found, retrieving from Wikipedia instead")
        r = requests.get(urls[pagename])
        # saving the list to a file
        f = open(pagepath, "w")
        f.write(r.text)
        soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [3]:
# Getting a list of heirarchical relationships between courts, from Wikipedia.

soup = loadWikiPage("USFedCircuitComposition")

fedTable = soup.table


In [4]:
# Creating the appealsTo dict of heirarchical relationships between courts, from the Wikipedia data.

# import pprint

appealsTo = {}

for line in fedTable.find_all(['p', 'li']):
    if line.a.get('title') != None:
        title = line.a.get('title').strip()
        if title.startswith('United States Court of Appeals') and title.endswith('Circuit'):
            circuit = title
        if title == circuit:
            appealsTo[title] = "Supreme Court of the United States"
        else:
            appealsTo[title] = circuit
            
# pprint.pprint(appealsTo)
                

In [5]:
import time

ordinals = ("first","second","third","fourth","fifth","sixth","seventh","eighth","ninth","tenth","eleventh")

def loadFJCPage(ordinal):

    pagename = 'federal-judicial-circuits-{0}-circuit'.format(ordinal)
    pagepath = './html/' + pagename
    
    try:
        # getting list of federal court circuits and states from file
        f = open(pagepath)
        page = f.read()
        assert len(page) > 100
        soup = BeautifulSoup(page, 'html.parser')

    except:
        print("List of federal court circuits and districts not found, retrieving from fjc.gov instead")
        url = "https://www.fjc.gov/history/administration/" + pagename
        r = requests.get(url, timeout=5)
        # saving the list to a file
        f = open(pagepath, "w")
        f.write(r.text)
        soup = BeautifulSoup(r.text, 'html.parser')
        time.sleep(1)
    return soup


In [6]:
'1801: Maine, Massachusetts, New Hampshire, Rhode Island. (2 Stat. 89)'.split(": ")

['1801', 'Maine, Massachusetts, New Hampshire, Rhode Island. (2 Stat. 89)']

In [7]:
import re
from datetime import datetime
import dateutil

circuitHistory = {}

def formatState(s):
    s = s.strip().replace("District of ", "").replace("Territory of", "").replace(" the", "")\
    .replace(" (encompassing the Territory Northwest of the Ohio and the Indiana Territory)", "")
    return s
    
def parseCircuitList(s):
    """
    Parses lists of states' judicial circuit assignments, scraped from
    https://www.fjc.gov/history/administration/federal-judicial-circuits.
    Output is a list of years with the corresponding circuit the court was assigned
    to in that year.
    """
    
    year = s.split(": ")
    states = year[1].split(". (")
    year = dateutil.parser.parse(year[0] + "-01-01") # TODO: add the correct dates, not January 1
    citation = re.search('(\d+ Stat\. \d+)', states[1])
    states = [formatState(state) for state in states[0].split(",")]
    return [(state, year) for state in states]
    
for o in ordinals:
    soup = loadFJCPage(o)
    a = soup.find_all('li', attrs={'class': None}, string=re.compile(":"))
    for line in a:
        entries = parseCircuitList(line.contents[0].string)
        for entry in entries:
            if entry[0] not in circuitHistory:
                circuitHistory[entry[0]] = {}
            circuitHistory[entry[0]][entry[1]] = o.capitalize()
        
print(circuitHistory)

{'Maine': {datetime.datetime(1801, 1, 1, 0, 0): 'First', datetime.datetime(1820, 1, 1, 0, 0): 'First', datetime.datetime(1915, 1, 1, 0, 0): 'First'}, 'Massachusetts': {datetime.datetime(1801, 1, 1, 0, 0): 'First', datetime.datetime(1802, 1, 1, 0, 0): 'First', datetime.datetime(1820, 1, 1, 0, 0): 'First', datetime.datetime(1915, 1, 1, 0, 0): 'First'}, 'New Hampshire': {datetime.datetime(1801, 1, 1, 0, 0): 'First', datetime.datetime(1802, 1, 1, 0, 0): 'First', datetime.datetime(1820, 1, 1, 0, 0): 'First', datetime.datetime(1915, 1, 1, 0, 0): 'First'}, 'Rhode Island': {datetime.datetime(1801, 1, 1, 0, 0): 'First', datetime.datetime(1802, 1, 1, 0, 0): 'First', datetime.datetime(1820, 1, 1, 0, 0): 'First', datetime.datetime(1915, 1, 1, 0, 0): 'First'}, 'Puerto Rico': {datetime.datetime(1915, 1, 1, 0, 0): 'First'}, 'Connecticut': {datetime.datetime(1801, 1, 1, 0, 0): 'Second'}, 'New York': {datetime.datetime(1801, 1, 1, 0, 0): 'Second'}, 'Vermont': {datetime.datetime(1801, 1, 1, 0, 0): 'Seco

In [8]:
# Turns the CourtListener format of federal court names into the format used by Wikipedia.

def longFedCircuitName(fullName):
    if fullName.startswith("Court of Appeals"):
        fullName = fullName.replace("Court of Appeals", "United States Court of Appeals", 1)
    fullName = fullName.replace("D.C.", "District of Columbia")
    return fullName

def longFedDistrictName(fullName):
    fullName = fullName.strip(".").replace(",", " for the").replace("D.", "District of")\
        .replace("C.", "Central ").replace("M.", "Middle ").replace("U.S.", "United States")
    fullName = fullName.replace("E.", "Eastern ").replace("W.", "Western ")\
        .replace("S.", "Southern ").replace("N.", "Northern ")
    fullName = fullName.replace('District Court for the Orleans',
                                'District Court for the District of Orleans')
    fullName = fullName.replace('District Court for the Virgin Islands', "District Court of the Virgin Islands")
    if fullName.endswith("Guam"):
        return "District Court of Guam"
    if not fullName.endswith(("Northern Mariana Islands","Guam","Virgin Islands")):
        if fullName.startswith("District Court"):
            fullName = fullName.replace("District Court", "United States District Court", 1)
    if fullName.startswith("Merit Systems"):
        fullName = fullName.replace("Merit Systems", "United States Merit Systems", 1)
    return fullName

def longFedCourtName(fullName, j):
    assert j in ('F', 'FD', 'FS')
    if j in ('FD', 'FS'):
        return longFedDistrictName(fullName)
    if j == 'F':
        return longFedCircuitName(fullName)

longFedCourtName('District Court, N.D. Mississippi', 'FD')

'United States District Court for the Northern District of Mississippi'

In [9]:
import json, os

foldername = "./courts-all/"

courts = []

for dirpath, dirnames, filenames in os.walk(foldername):
    for filename in filenames:
        source = foldername + filename
        markup = open(source)
        court = json.load(markup)
        if "jurisdiction" in court:
            if court['citation_string'] in ('Mo. S.D.', 'C.D. Mo.'):
                court['jurisdiction'] = 'SA' # correcting an error in Courtlistener data, they're state courts.
            if court['jurisdiction'] in ('F', 'FD', 'FS'): # will need to import other kinds of courts later
                courts.append(court)

cd = {}
                
for court in courts:
    cd[longFedCourtName(court['full_name'], court['jurisdiction'])] = court


In [10]:
cd["United States District Court for the Southern District of Texas"]

{'citation_string': 'S.D. Tex.',
 'date_modified': '2013-10-22T15:56:25.488000Z',
 'end_date': None,
 'full_name': 'District Court, S.D. Texas',
 'has_opinion_scraper': False,
 'has_oral_argument_scraper': False,
 'in_use': True,
 'jurisdiction': 'FD',
 'position': 250.5,
 'resource_uri': 'http://www.courtlistener.com/api/rest/v3/courts/txsd/',
 'short_name': 'S.D. Texas',
 'start_date': None,
 'url': 'http://www.txs.uscourts.gov/'}

In [11]:
# importing these abbreviations to generate abbreviated names for courts that don't have them.

import json

with open("state_abbreviations.json") as json_data:
    sd = json.load(json_data)
    json_data.close()
    

In [12]:
def make_citation_string(name, sd):
    for state in sd:
        name = name.replace(state, sd[state]['AP'])
    return name

make_citation_string("D. Mississippi", sd)

'D. Miss.'

In [13]:
for i in cd:
    if cd[i]["citation_string"] == "":
        cd[i]["citation_string"] = make_citation_string(cd[i]["short_name"], sd)


In [14]:
defunctSoup = loadWikiPage("List_of_former_United_States_district_courts")

In [15]:
courtStates = []

headings = defunctSoup.find_all("h2")
for heading in headings[:-3]: # The later headings aren't states
    if heading.span:
        courtStates.append(heading.span.contents[0])

print(courtStates)        

['Alabama', 'Arkansas', 'California', 'Florida', 'Georgia', 'Illinois', 'Indiana', 'Iowa', 'Kentucky', 'Louisiana', 'Michigan', 'Mississippi', 'Missouri', 'New Jersey', 'New York', 'North Carolina', 'Ohio', 'Pennsylvania', 'South Carolina', 'Tennessee', 'Texas', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin']


In [16]:
def getCourtDates(table):
    """
    Returns a tuple with a defunct court's start and end dates from a table scraped from
    https://en.wikipedia.org/wiki/List_of_former_United_States_district_courts. Assumes that the beginning of the
    first judge's service is the start date and end of the last judge's service is the end date.
    """
    startcolumn = None
    header = "td"
    for i in range(4):
        if str(table.find_all(header)[i].contents[0]).__contains__("Began active"):
            startcolumn = i
    if startcolumn:
        rows = table.find_all("tr")[1:]
        startdates = [dateutil.parser.parse(row.find_all("td")[startcolumn].find_all("span")[1].contents[0]) for row in rows]
        enddates = [dateutil.parser.parse(row.find_all("td")[startcolumn + 1].find_all("span")[1].contents[0]) for row in rows]
        return (min(startdates), max(enddates), )
    else:
        # print(str(table.find_all(header)[i].contents[0]))
        return None

def getCourtName(table, stateList):
    name = None
    links = table.parent.find_all("a")
    i = 0
    while name == None:
        if links[i].contents != []:
            if links[i].contents[0] in stateList:
                name = links[i].contents[0]
        i += 1
    return name

courtDates = {}

tables = defunctSoup.find_all("table")
for table in tables:
    dates = getCourtDates(table)
    if dates != None:
        if getCourtName(table, courtStates) != "South Carolina": # Court by this name isn't defunct
            courtDates["United States District Court for the District of " + getCourtName(table, courtStates)] = {
            "start_date": datetime.strftime(dates[0], '%Y-%m-%d'),
            "end_date": datetime.strftime(dates[1], '%Y-%m-%d')    
            }

In [17]:
for court in courtDates.keys():
    if court in cd.keys():
        if cd[court]["start_date"] == "" or "None" or None:
            cd[court]["start_date"] = courtDates[court]["start_date"]
        if cd[court]["end_date"] == "" or "None" or None:
            cd[court]["end_date"] = courtDates[court]["end_date"]
    else:
        print(str(court + " not found"))

United States District Court for the District of Arkansas not found


In [18]:
# Adding info on defunct courts not covered by the Wikipedia table scraped above


defunctAppeals = {
                 # "United States District Court for the District of California": "Supreme Court of the United States",
                 "Emergency Court of Appeals": "Supreme Court of the United States",
                 'Court of Customs and Patent Appeals': "Supreme Court of the United States",
                 'Temporary Emergency Court of Appeals': "Supreme Court of the United States",
                 'United States Customs Court': 'United States Court of Appeals for the Federal Circuit',
                 'Commerce Court': "Supreme Court of the United States",
                 "United States Court of Claims": "Supreme Court of the United States",
                 "Special Court under the Regional Rail Reorganization Act": "Supreme Court of the United States",
                 "United States District Court for the District of Orleans": "Supreme Court of the United States"
                }
                 
                 # 'United States District Court for the District of Mississippi': "Supreme Court of the United States",
                 # the District of Missouri was assigned to the Eighth Circuit on March 3, 1837.
                 # "United States District Court for the District of Missouri": "United States Court of Appeals for the Eighth Circuit",
                 # "United States District Court for the District of New York": "United States Court of Appeals for the Second Circuit"}

# Non-defunct courts not listed in the Wikipedia table for whatever reason.

otherAppeals = {"Navy-Marine Corps Court of Criminal Appeals": "Court of Appeals for the Armed Forces",
                "Army Court of Criminal Appeals": "Court of Appeals for the Armed Forces",
                "United States Air Force Court of Criminal Appeals": "Court of Appeals for the Armed Forces",
                "United States Court of Military Commission Review": 'United States District Court for the District of Columbia',
                "Board of Veterans' Appeals": "United States Court of Appeals for Veterans Claims",
                "Foreign Intelligence Surveillance Court": "Foreign Intelligence Surveillance Court of Review",
                "Foreign Intelligence Surveillance Court of Review": "Supreme Court of the United States",
                # The Tax Court's appeals don't go to the Supreme Court, but to any geographic circuit.
                # I don't know whether it has to follow the law of the litigant's circuit, but probably so.
                "United States Tax Court": "Supreme Court of the United States",
                "Court of Appeals for the Armed Forces": "Supreme Court of the United States"}

for item in defunctAppeals:
    cd[item]['reviewed_by'] = defunctAppeals[item]

for item in otherAppeals:
    cd[item]['reviewed_by'] = otherAppeals[item]

In [19]:
appealsTo

{'Armed Services Board of Contract Appeals': 'United States Court of Appeals for the Federal Circuit',
 'Bureau of Justice Assistance': 'United States Court of Appeals for the Federal Circuit',
 'Civilian Board of Contract Appeals': 'United States Court of Appeals for the Federal Circuit',
 'District Court for the Northern Mariana Islands': 'United States Court of Appeals for the Ninth Circuit',
 'District Court of Guam': 'United States Court of Appeals for the Ninth Circuit',
 'District Court of the Virgin Islands': 'United States Court of Appeals for the Third Circuit',
 'Government Accountability Office': 'United States Court of Appeals for the Federal Circuit',
 'Patent Trial and Appeal Board': 'United States Court of Appeals for the Federal Circuit',
 'Trademark Trial and Appeal Board': 'United States Court of Appeals for the Federal Circuit',
 'United States Congress Office of Compliance': 'United States Court of Appeals for the Federal Circuit',
 'United States Court of Appeals 

In [20]:
cd["Supreme Court of the United States"]

{'citation_string': 'SCOTUS',
 'date_modified': '2014-10-31T01:59:15.952000Z',
 'end_date': None,
 'full_name': 'Supreme Court of the United States',
 'has_opinion_scraper': True,
 'has_oral_argument_scraper': True,
 'in_use': True,
 'jurisdiction': 'F',
 'position': 1.0,
 'resource_uri': 'http://www.courtlistener.com/api/rest/v3/courts/scotus/',
 'short_name': 'Supreme Court',
 'start_date': '1789-09-24',
 'url': 'http://supremecourt.gov/'}

In [21]:
from typing import Dict

def getLastAppellate(closure: datetime, history: Dict[datetime, str]) -> datetime:
    """
    Take the datetime "closure", look up the latest datetime key in "history" that's not later than closure,
    and return the value for that key.
    """
    if closure <= min(history):
        return 'Supreme Court of the United States'
    else:
        return "United States Court of Appeals for the {0} Circuit".format(history[max(year for year in history if year < closure)])

getLastAppellate(datetime(1837, 1, 1, 0, 0), circuitHistory["Illinois"])

'Supreme Court of the United States'

In [22]:
# This is failing because I don't have a way to handle the defunct courts
# A comprehensive solution would capture the date range that every court appealed to every other one...
# TODO: add WikiData URIs for each court
    
for court in cd:
    if 'reviewed_by' not in cd[court].keys():
        if court in ('Supreme Court of the United States', 'United States Attorneys General'):
            cd[court]['reviewed_by'] = None
        elif court.startswith("U."): # Only intended to catch circuit courts from before they were numbered
            cd[court]['reviewed_by'] = 'Supreme Court of the United States'
        elif court in appealsTo:
            cd[court]['reviewed_by'] = appealsTo[court]
        else:
            for length in range(1, 4): # Will overwrite Virginia if West Virginia is also in the list.
                stateName = " ".join(court.rsplit(" ", length)[-1 * length:])
                if stateName in circuitHistory:
                    assert cd[court]['end_date'] != None
                    cd[court]['reviewed_by'] = getLastAppellate(dateutil.parser.parse(cd[court]['end_date']), \
                                                                circuitHistory[stateName])

In [23]:
type(cd['Supreme Court of the United States']['start_date'])

str

In [35]:
# I made up this abbreviation

cd['Circuit Court for California']["citation_string"] = "Cal. Cir."

# Fixing an apparent Courtlistener data error

cd['United States Circuit Court for the District of Tennessee']["citation_string"] = "Cir. Ct. Tenn."


In [25]:
for i in cd:
    if cd[i]["citation_string"] == "":
        print(i)

In [26]:
from graphviz import Digraph

dot = Digraph(comment='Federal Courts')
for i in cd:
    dot.node(i, cd[i]["citation_string"])
for i in cd:
    try: 
        if cd[i]['reviewed_by'] != None:
            dot.edge(i, cd[i]['reviewed_by'])
    except: 
        print(cd[i])

In [27]:
# Makes a very ugly chart

dot.render(view=True)

'Digraph.gv.pdf'

In [30]:
print(dot)

// Federal Courts
digraph {
	"United States District Court for the Western District of Texas" [label="W.D. Tex."]
	"United States District Court for the Northern District of Indiana" [label="N.D. Ind."]
	"United States District Court for the Southern District of Georgia" [label="S.D. Ga."]
	"United States District Court for the District of California" [label="D. Cal."]
	"United States District Court for the District of Alaska" [label="D. Alaska"]
	"United States District Court for the District of Connecticut" [label="D. Conn."]
	"United States District Court for the District of Colorado" [label="D. Colo."]
	"United States District Court for the Middle District of Pennsylvania" [label="M.D. Penn."]
	"United States Merit Systems Protection Board" [label=MSPB]
	"United States Court of Appeals for the Tenth Circuit" [label="10th Cir."]
	"Emergency Court of Appeals" [label="Emer. Ct. App."]
	"United States District Court for the District of Delaware" [label="D. Del."]
	"United States Distri